# Load Library

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', 50)
import pandas_profiling as pf
import numpy as np

from sklearn.preprocessing import OrdinalEncoder

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

import seaborn as sns
import matplotlib.pyplot as plt

from imblearn.over_sampling import SMOTE

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from xgboost.sklearn import XGBClassifier
from sklearn.metrics import f1_score

import optuna

# Load Data

In [ ]:
df = pd.read_csv('Train.csv')

In [ ]:
df_test = pd.read_csv('Test.csv')

# Data Understanding

In [ ]:
full_profile_train = pf.ProfileReport(df, title='Profiling Report', html={'style':{'full_width':True}})

In [ ]:
full_profile_train.to_notebook_iframe()

# Data Processing

In [ ]:
oe = OrdinalEncoder()

In [ ]:
# Encoding categorical feature into numerical feature
df['housing'] = oe.fit_transform(df[['housing']])
df['payment'] = oe.fit_transform(df[['payment']])
df['source_of_application'] = oe.fit_transform(df[['source_of_application']])
df['employment'] = oe.fit_transform(df[['employment']])
df['os'] = oe.fit_transform(df[['os']])

In [ ]:
# Encoding categorical feature into numerical feature
df_test['housing'] = oe.fit_transform(df_test[['housing']])
df_test['payment'] = oe.fit_transform(df_test[['payment']])
df_test['source_of_application'] = oe.fit_transform(df_test[['source_of_application']])
df_test['employment'] = oe.fit_transform(df_test[['employment']])
df_test['os'] = oe.fit_transform(df_test[['os']])

In [ ]:
# Replace -1 with np.nan in address_months_count and  feature 
df['address_months_count'].replace(to_replace=-1, value=np.nan, inplace=True)
df['current_address_months_count'].replace(to_replace=-1, value=np.nan, inplace=True)

df['bank_months'].replace(to_replace=-1, value=np.nan, inplace=True)
df['distinct_device_emails'].replace(to_replace=-1, value=np.nan, inplace=True)
df['session_length'].replace(to_replace=-1, value=np.nan, inplace=True)

In [ ]:
# Replace -1 with np.nan in address_months_count and  feature 
df_test['address_months_count'].replace(to_replace=-1, value=np.nan, inplace=True)
df_test['current_address_months_count'].replace(to_replace=-1, value=np.nan, inplace=True)

# tackling other untold features that have -1 :). when they already have NaN
df_test['bank_months'].replace(to_replace=-1, value=np.nan, inplace=True)
df_test['distinct_device_emails'].replace(to_replace=-1, value=np.nan, inplace=True)
df_test['session_length'].replace(to_replace=-1, value=np.nan, inplace=True)

In [ ]:
df.info()

# Data Understanding II

In [ ]:
full_profile_train = pf.ProfileReport(df, title='Profiling Report', html={'style':{'full_width':True}})

In [ ]:
full_profile_train.to_notebook_iframe()

# Data Processing II

## Drop columns

In [ ]:
df.drop(columns=['id'], inplace=True)

## Drop any duplicated values

In [ ]:
df.shape

In [ ]:
# drop duplicated rows
df.drop_duplicates(inplace=True)

In [ ]:
# drop duplicated rows
df_test.drop_duplicates(inplace=True)

In [ ]:
df.shape

## Impute missing value

In [ ]:
imputer = IterativeImputer()

In [ ]:
df.loc[:, df.columns != 'fraud'] = imputer.fit_transform(df.loc[:, df.columns != 'fraud'])

In [ ]:
df_test.loc[:, df_test.columns != 'id'] = imputer.fit_transform(df_test.loc[:, df_test.columns != 'id'])

## Assess imblanced data

In [ ]:
sns.countplot(x=df['fraud'], data=df)
plt.title("Distribution of the label")
plt.show()

In [ ]:
X = df.loc[:,  df.columns != 'fraud']
y = df[['fraud']]

In [ ]:
sm = SMOTE(random_state=24)
X_res, y_res = sm.fit_resample(X,y)

# Modeling

## Build baseline model

In [ ]:
# Create function to automate model training
def train_model(X_train, y_train):
    # list of base_estimator 
    base_estim = [LogisticRegression(), DecisionTreeClassifier(), XGBClassifier()] 
    # Create empty list to store f1_score
    f1score = []

    # Loop through list of models
    for model in base_estim:
        estimator = model
        estimator.fit(X_train, y_train)
        y_pred = estimator.predict(X_train)

        f1score.append(f1_score(y_train, y_pred))
       
    # Create dictionary to store scores
    result = {
        "Model" : [LogisticRegression, DecisionTreeClassifier, XGBClassifier],
        "f1_score" : f1score
    }

    # Return dataframe
    return pd.DataFrame(result)

In [ ]:
table = train_model(X_res, y_res)
table

## Chosen baseline model

In [ ]:
model = XGBClassifier()
model.fit(X_res, y_res)
y_pred = model.predict(X_res)
print(f1_score(y_res, y_pred))

In [ ]:
model.get_params()

# Improvement Model

In [ ]:
def objectiverf(trial, X_train, y_train):
    
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 90, 200), # number of tree in enesemble, risk of overfitting and complexity of problem (100)
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0, 1), # control randomness (0, 1]
        'subsample' : trial.suggest_float('subsample', 0.5, 1), # >= 0.5 [0,1]
        'learning_rate': trial.suggest_float('learning_rate', 0, 1), # preventing overfitting [0,1]
        'max_depth': trial.suggest_int('max_depth', 6, 14), # semakin tinggi semakin overfitting [0, ...] default=6
        'alpha': trial.suggest_float('alpha', 0, 3), # increasing this will make model more convservative [0, ...]
        'gamma': trial.suggest_float('gamma', 0, 3), # increasing this will make model more convservative [0, ...] 
        'min_child_weight' : trial.suggest_int('min_child_weight', 3, 6) # prevent overfitting and control the trade-off between complexity and generalization [0, ...] 
    }

    clf = XGBClassifier(n_estimators=params['n_estimators'],
                                  colsample_bytree=params['colsample_bytree'],
                                  subsample=params['subsample'],
                                  learning_rate=params['learning_rate'],
                                  max_depth=params['max_depth'],
                                  alpha=params['alpha'], 
                                  gamma=params['gamma'], 
                                  min_child_weight=params['min_child_weight'],
                                  booster='gbtree', 
                                  random_state=42)

    model = clf.fit(X_train, y_train)
    
    y_pred = model.predict(X_train)
    f1score = f1_score(y_res, y_pred)
    
    return f1score

In [ ]:
study2 = optuna.create_study(direction='maximize')
study2.optimize(lambda trial: objectiverf(trial, X_res, y_res), n_trials=100)

# Print the best hyperparameters and score
print('Best score:', study2.best_value)
print('Best parameters:', study2.best_params)

In [ ]:
# set the hyperparameters for the model

params =  {'n_estimators': 160, 'colsample_bytree': 0.732657163298562, 'subsample': 0.9112391152638936, 'learning_rate': 0.32238830328241286, 'max_depth': 13, 'alpha': 1.761724762047104, 'gamma': 1.2019463459946005, 'min_child_weight': 4}
clf = XGBClassifier(n_estimators=params['n_estimators'],
                              colsample_bytree=params['colsample_bytree'],
                              subsample=params['subsample'],
                              learning_rate=params['learning_rate'],
                              max_depth=params['max_depth'],
                              alpha=params['alpha'], 
                              gamma=params['gamma'], 
                              min_child_weight=params['min_child_weight'],
                              booster='gbtree', #for medium-size dataset
                              random_state=42) 

model2 = XGBClassifier(**params) 
model2.fit(X_res, y_res)

# Make predictions on the testing data
y2_pred = model2.predict(df_test.loc[: ,df_test.columns != 'id'])

# 6. Submission

In [ ]:
final_result = {
    'id': df_test['id'],
    'fraud' : y2_pred
}
final = pd.DataFrame(final_result)
final

In [ ]:
final.loc[final['fraud'] == 0, 'fraud'] = 'Not Fraud'
final.loc[final['fraud'] == 1, 'fraud'] = 'Fraud'

In [ ]:
final.to_csv('C:\Data\prakkomstat8.csv')